In [ ]:
# check for convergence

# calculate single derivative

# calculate all derivatives for one atom

# calculate derivatives for whole molecule

In [ ]:
# check number of electrons 

In [ ]:
# error for all points?
# error as function of gpts
# energy UEG as function of boxsize/volume

In [1]:
import sys
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/hitp/')
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')

import cpmd_io
import prepare_calculations as pc
import utils_qm as uqm
import numpy as np
import collections
import os

In /home/sahre/opt/miniconda3/envs/qm-tools/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sahre/opt/miniconda3/envs/qm-tools/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sahre/opt/miniconda3/envs/qm-tools/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/sahre/opt/miniconda3/envs/qm-tools/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor rele

In [2]:
def paths2logfiles(comp, delta_lambda, lam):
    """
    prepare the run dirs for finite difference partial derivatives for small molecules
    comp: name of compound
    delta_lambda: delta for finite difference
    lam: lambda_value for which derivatives will be calculated
    """
    all_paths = [] # paths to run dirs
    # read optimized geometry and extract molecule specific parameters
    xyz_path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/GEOMETRY.xyz'
    atom_symbols, nuc_charges, positions, valence_charges = pc.parse_xyz_for_CPMD_input(xyz_path)


    for i, atom in enumerate(atom_symbols['elIdx']):
        for fd in ['bw', 'fw']:
            path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/lam_{np.round(lam, 3)}/{atom}/{fd}/run.log'
            all_paths.append(path)
            # define the array of lambda values
            lambda_values = np.full_like(valence_charges, fill_value=lam, dtype='float')
            if fd == 'bw':
                lambda_values[i] -= delta_lambda
            elif fd == 'fw':
                lambda_values[i] += delta_lambda
            else:
                raise ValueError('Unknown finite difference operation')
    return(all_paths)


def check_convergence(logfile, conv_thresh):
    status = 'not converged'
    iteration_start = 0
    for i, line in enumerate(logfile):
        if 'NFI' in line:
            iteration_start = i +1
            break

    j = 0

    for line in logfile[iteration_start:]:
        line_splitted = line.split()

        try:
            gemax = float(line_splitted[1])
        except:
            gemax = conv_thresh + 10

        if gemax < conv_thresh:
            status = 'converged'
            break
    return(status)

def get_num_ve(logfile):
    ve_g = 0
    ve_r = 0
    for line in logfile:
        if "IN G-SPACE =" in line:
            ve_g = float(line.strip('\n').split()[-1])
        elif "IN R-SPACE =" in line:
            ve_r = float(line.strip('\n').split()[-1])
    
    return(ve_g, ve_r)

In [3]:
conv_thresh = 1e-6
lambda_values = np.round(np.arange(0, 1.05, 0.05), 3)
compounds = ['CH4', 'H2O','NH3', 'HF', 'ethane', 'ethene', 'ethyne', 'HCN']
status = {k:[] for k in compounds}
num_ve = {k:[] for k in compounds}
for comp in compounds:
    for lam in lambda_values:
        path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/lam_{lam}/run.log'
        with open(path, 'r') as f:
            logfile = f.readlines()
        status[comp].append([lam, check_convergence(logfile, conv_thresh)])
        
        ve_g, ve_r = get_num_ve(logfile)
        num_ve[comp].append([ve_g, ve_r])

In [4]:
correct_ve = []
for k in num_ve.keys():
    correct_ve.append(num_ve[k][-1][0])
    
for k, cve in zip(num_ve.keys(), correct_ve):
    for l, l1 in zip(num_ve[k], lambda_values):
        if np.isclose(l[0], cve) and np.isclose(l[1], cve):
            pass
        else:
            print(k +" " + str(l1))

In [ ]:
correct_ve = [8,8,8,8, 'ethane', 'ethene', 'ethyne', 'HCN']

In [6]:
for k in status.keys():
    for i in status[k]:
        if i[1] == 'not converged':
            print(f'{k} {i[0]} {i[1]}')

In [ ]:
paths = uqm.read_list('/data/sahre/projects/finite_differences/small_molecules/fd_rest')

In [ ]:
for p in paths:
    logpath = os.path.join(p, 'run.log')
    if os.path.isfile(logpath):
        with open(logpath, 'r') as f:
            logfile = f.readlines()
        conv_thresh = 1e-6
        if not check_convergence(logfile, conv_thresh) == 'converged':
            print(f'Not converged {p}')

        g, r = get_num_ve(logfile)
        assert np.isclose(g,r)
        if 'ethane' in p:
            assert np.isclose(g, 14)
        elif 'ethyne' in p:
            assert np.isclose(g, 10)
        elif 'HCN' in p:
            assert np.isclose(g, 10)
        elif 'ethene' in p:
            assert np.isclose(g, 12)
        # if not np.isclose(nve, g) and not np.isclose(nve,r):
        #     print(f'Wrong num elec {p}')
    else:
        print(f'Missing {p}')

In [ ]:
paths

In [ ]:
num_ve

In [7]:
for k in status.keys():
    for i in status[k]:
        if i[1] == 'not converged':
            print(f'{k} {i[0]} {i[1]}')

In [8]:
def df_paths(comp, delta_lambda, lam):
    """
    prepare the run dirs for finite difference partial derivatives for small molecules
    comp: name of compound
    delta_lambda: delta for finite difference
    lam: lambda_value for which derivatives will be calculated
    """
    all_paths = [] # paths to run dirs
    # path to PP-files and type of PP (pbe)
    pp_dir = '/data/sahre/PP_LIBRARY'
    pp_type = 'GH_PBE'

    # input templates
    template_inp = '/data/sahre/projects/atomic-energies/templates_inp/cpmd_template_restart.inp'
    template_inp_small_lambda = '/data/sahre/projects/atomic-energies/templates_inp/cpmd_template_small_lambda_restart.inp'

    # read optimized geometry and extract molecule specific parameters
    xyz_path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/GEOMETRY.xyz'
    atom_symbols, nuc_charges, positions, valence_charges = pc.parse_xyz_for_CPMD_input(xyz_path)

    # calculate correct boxsize
    num_ve = valence_charges.sum()
    reference_density = 8/(15**3) # density of UEG for a boxlength of 15 Ang and 8 valence electrons
    boxsize = (num_ve/reference_density)**(1/3)
    shift2center = True

    for i, atom in enumerate(atom_symbols['elIdx']):
        for fd in ['bw', 'fw']:
            path = f'/data/sahre/projects/finite_differences/small_molecules/{comp}/lam_{np.round(lam, 3)}/{atom}/{fd}'
            all_paths.append(path)
    return(all_paths)

In [9]:
base_lambda = np.arange(0.05, 1.05, 0.05)
compounds = ['CH4', 'NH3', "H2O",'HF', 'ethane', 'ethene', 'ethyne', 'HCN']
delta_lambda = 5e-5
paths = collections.defaultdict(list)
for comp in compounds:
    for lam in base_lambda:
        paths[comp].extend(df_paths(comp, delta_lambda, lam))

In [10]:
for k, nve in zip(paths.keys(),[8,8,8,8,14,12,10,10]):
    for p in paths[k]:
        logpath = os.path.join(p, 'run.log')
        if os.path.isfile(logpath):
            with open(logpath, 'r') as f:
                logfile = f.readlines()
            conv_thresh = 1e-6
            if not check_convergence(logfile, conv_thresh) == 'converged':
                print(f'Not converged {p}')
                
            g, r = get_num_ve(logfile)
            if not np.isclose(nve, g) and not np.isclose(nve,r):
                print(f'Wrong num elec {p}')
        else:
            print(f'Missing {p}')
    
        

# UEG

In [ ]:
ves = ['ve_8', 've_10', 've_12', 've_14']
boxsizes = [15, 20, 22.5, 25]
energy_ve = dict()
for ve in ves:
    energy_ve[ve] = []
    for boxsize in boxsizes:
        path_log = f'/data/sahre/projects/finite_differences/small_molecules/UEG/{ve}/a_{boxsize}/run.log'
        with open(path_log, 'r') as f:
            logfile = f.readlines()
        conv_thresh = 1e-6
        status = check_convergence(logfile, conv_thresh)
        if status == 'not converged':
            print(path_log)
        energy_ve[ve].append(cpmd_io.get_energy_contribution(logfile, 'TOTAL ENERGY ='))

In [ ]:
plt.rcParams['font.size'] = 15
for k in energy_ve.keys():
    plt.plot(boxsizes, np.array(energy_ve[k])/float(k.split('_')[1]), '-o',label = k)
plt.legend(bbox_to_anchor = (1.05,1))
plt.xlabel(r'a (Ang)')
plt.ylabel(r'E UEG (Ha)')

In [ ]:
np.array(energy_ve[k])/float(k.split('_')[1])

In [ ]:
-0.41775901/10 - (-0.33797308/8)



In [ ]:
energy_ve

In [ ]:
plt.rcParams['font.size'] = 15
for k in energy_ve.keys():
    plt.plot(float(k.split('_')[1])/np.array(boxsizes)**3, np.array(energy_ve[k])/float(k.split('_')[1]), '-o',label = k)
plt.legend(bbox_to_anchor = (1.05,1))
plt.xlabel(r'$\rho$ (1/Ang^3)')
plt.ylabel(r'E UEG (Ha/e)')

In [ ]:
boxsizes